In [16]:
import requests 
from bs4 import BeautifulSoup 
r = requests.get('https://www.camclarkfordrichmond.com/used/') 
print(r) 
soup = BeautifulSoup(r.content, 'html.parser') 
print(soup.prettify()) 

<Response [200]>
<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">
<html lang="en">
 <!--[if IE 7]> <html class="vd-list ie ie7" lang="en-US"> <![endif]-->
 <!--[if IE 8]> <html class="vd-list ie ie8" lang="en-US"> <![endif]-->
 <!--[if IE 9]> <html class="vd-list ie ie9" lang="en-US"> <![endif]-->
 <!--[if !(IE 7) | !(IE 8) | !(IE 9) ]><!-->
 <html lang="en" xml:lang="en" xmlns="http://www.w3.org/1999/xhtml">
  <!--<![endif]-->
  <head>
   <meta content="IE=edge" http-equiv="X-UA-Compatible"/>
   <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
   <meta content="index, follow" name="robots"/>
   <meta content="text/html; charset=utf-8" http-equiv="Content-Type">
    <meta content="width=device-width, initial-scale=1" name="viewport"/>
    <title>
     Used Cars, SUVs, Trucks for Sale in Richmond | Cam Clark Ford Richmond
    </title>
    <meta content="Shop our selection of used vehicles for sa

In [17]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
import time
import pandas as pd

In [18]:
# Setup Selenium WebDriver
options = Options()
options.headless = True  # Run headless Chrome
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=options)

# URL of the website to scrape
url = "https://www.camclarkfordrichmond.com/used/pg/3"
driver.get(url)

# Wait for the page to load
time.sleep(5)  # Adjust the sleep time if necessary

# Scroll to the bottom of the page to load all dynamic content
last_height = driver.execute_script("return document.body.scrollHeight")
while True:
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(3)  # Wait for new content to load
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height

# Find the container with car listings
car_listings = driver.find_elements(By.CLASS_NAME, 'vehicle-list-cell')

# Extract individual car details
cars = []
for item in car_listings:
    # Extract name
    try:
        name_tag = item.find_element(By.CLASS_NAME, 'vehicle-year-make-model')
        year = name_tag.find_element(By.XPATH, ".//span[@itemprop='releaseDate']").text.strip()
        manufacturer = name_tag.find_element(By.XPATH, ".//span[@itemprop='manufacturer']").text.strip()
        model = name_tag.find_element(By.XPATH, ".//span[@itemprop='model']").text.strip()
        name = f"{year} {manufacturer} {model}"
    except:
        name = "N/A"
    
    # Extract price
    try:
        price_tag = item.find_element(By.CSS_SELECTOR, 'span[itemprop="price"]')
        price = price_tag.text.strip()
    except:
        price = "N/A"
    
    cars.append({'name': name, 'price': price})

# Close the WebDriverfrom selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
import time

# Setup Selenium WebDriver
options = Options()
options.headless = True  # Run headless Chrome
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=options)

# URL of the website to scrape
url = "https://www.camclarkfordrichmond.com/used/"
driver.get(url)

# Wait for the page to load
time.sleep(5)  # Adjust the sleep time if necessary

# Scroll to the bottom of the page to load all dynamic content
last_height = driver.execute_script("return document.body.scrollHeight")
while True:
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(3)  # Wait for new content to load
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height

# Find all car listing elements
car_listings = driver.find_elements(By.CLASS_NAME, 'vehicle-list-cell')

# Extract individual car details
cars = []
for item in car_listings:
    name_tag = item.find_element(By.CLASS_NAME, 'vehicle-year-make-model')
    year = name_tag.find_element(By.XPATH, ".//span[@itemprop='releaseDate']").text.strip()
    manufacturer = name_tag.find_element(By.XPATH, ".//span[@itemprop='manufacturer']").text.strip()
    model = name_tag.find_element(By.XPATH, ".//span[@itemprop='model']").text.strip()
    name = f"{year} {manufacturer} {model}"
    

    price_tag = item.find_element(By.CSS_SELECTOR, 'span[itemprop="price"]')
    price = price_tag.text.strip()
    
    cars.append({'name': name, 'price': price})

# Print the extracted car details
if cars:
    for car in cars:
        print(f"Name: {car['name']}, Price: {car['price']}")
else:
    print("No car data extracted.")

# Close the WebDriver
driver.quit()

Name: 2015 Mazda CX-5 GT (A6) 4dr All-Wheel Drive, Price: $17,995
Name: 2016 Honda Civic LX (CVT) 4dr Sedan, Price: $21,995
Name: 2017 Mazda CX-3 GT 4dr All-Wheel Drive Sport Utility, Price: $25,995
Name: 2019 Ford Escape SE 4dr 4x4, Price: $23,995
Name: 2017 RAM ProMaster City SLT Cargo Van, Price: $26,995
Name: 2018 Nissan NV200 S 4dr Compact Cargo Van, Price: $25,995
Name: 2022 Hyundai Elantra Preferred w/Sun & Tech Pkg 4dr Sedan, Price: $26,995
Name: 2021 Toyota Corolla Hatchback 5dr, Price: $26,995
Name: 2022 Hyundai Kona 2.0L Preferred Sun & Leather Package 4dr All-Wheel Drive, Price: $28,995
Name: 2017 Jeep Grand Cherokee Trailhawk 4dr 4x4, Price: $28,995
Name: 2022 Honda Civic Touring 4dr Sedan, Price: $28,995
Name: 2022 Ford Escape SEL 4dr All-Wheel Drive, Price: $32,995
Name: 2021 Ford Escape SE Hybrid 4dr All-Wheel Drive, Price: $27,995
Name: 2017 Jaguar F-PACE 35t Prestige All-Wheel Drive, Price: $28,880
Name: 2017 Ford F-150, Price: $31,995
Name: 2019 Ford Mustang, Price: 

In [19]:
df = pd.DataFrame(cars)
print(df)

                                                 name     price
0         2015 Mazda CX-5 GT (A6) 4dr All-Wheel Drive   $17,995
1                 2016 Honda Civic LX (CVT) 4dr Sedan   $21,995
2   2017 Mazda CX-3 GT 4dr All-Wheel Drive Sport U...   $25,995
3                         2019 Ford Escape SE 4dr 4x4   $23,995
4               2017 RAM ProMaster City SLT Cargo Van   $26,995
5           2018 Nissan NV200 S 4dr Compact Cargo Van   $25,995
6   2022 Hyundai Elantra Preferred w/Sun & Tech Pk...   $26,995
7                   2021 Toyota Corolla Hatchback 5dr   $26,995
8   2022 Hyundai Kona 2.0L Preferred Sun & Leather...   $28,995
9          2017 Jeep Grand Cherokee Trailhawk 4dr 4x4   $28,995
10                 2022 Honda Civic Touring 4dr Sedan   $28,995
11           2022 Ford Escape SEL 4dr All-Wheel Drive   $32,995
12     2021 Ford Escape SE Hybrid 4dr All-Wheel Drive   $27,995
13    2017 Jaguar F-PACE 35t Prestige All-Wheel Drive   $28,880
14                                    20